In [1]:
cd ..

/Users/johnphillips/Desktop/DSI-Class-Stuff/BaseBall


In [2]:
%matplotlib inline

In [3]:
%run __init__.py

In [4]:
# Start with reading the data sets from the pickles

teams = pd.read_pickle('data/teams.p') # Full Data
teams_num = pd.read_pickle('data/teams_num.p') # Numeric Features
new_reduced = pd.read_pickle('data/new_reduced.p') # Reduced from PCA
salaries = pd.read_pickle('data/salaries.p') # Team Salaries 1985 - 2016
rockies = pd.read_pickle('data/rockies.p') # 2017 Rockies Team Stats

## Rare occurances

I heard there was once a World Series MVP from a non-winning Team.  How can we find out?

In [5]:
# Start with loading this csv
awards = pd.read_csv('data/AwardsPlayers.csv') 

In [6]:
awards.head()

,playerID,awardID,yearID,lgID,tie,notes
0,bondto01,Pitching Triple Crown,1877,NL,NaN,NaN
1,hinespa01,Triple Crown,1878,NL,NaN,NaN
2,heckegu01,Pitching Triple Crown,1884,AA,NaN,NaN
3,radboch01,Pitching Triple Crown,1884,NL,NaN,NaN
4,oneilti01,Triple Crown,1887,AA,NaN,NaN


In [7]:
# Let's see what awards there are.
awards.awardID.unique()

# We'll want to find 'World Series MVP'

array(['Pitching Triple Crown', 'Triple Crown',
       'Baseball Magazine All-Star', 'Most Valuable Player',
       'TSN All-Star', 'TSN Guide MVP',
       'TSN Major League Player of the Year', 'TSN Pitcher of the Year',
       'TSN Player of the Year', 'Rookie of the Year', 'Babe Ruth Award',
       'Lou Gehrig Memorial Award', 'World Series MVP', 'Cy Young Award',
       'Gold Glove', 'TSN Fireman of the Year', 'All-Star Game MVP',
       'Hutch Award', 'Roberto Clemente Award',
       'Rolaids Relief Man Award', 'NLCS MVP', 'ALCS MVP',
       'Silver Slugger', 'Branch Rickey Award', 'Hank Aaron Award',
       'TSN Reliever of the Year', 'Comeback Player of the Year',
       'Outstanding DH Award', 'Reliever of the Year Award'], dtype=object)

In [8]:
# Find only World Series MVP award winners
mvp_awards = awards[awards['awardID'] == 'World Series MVP']
mvp_awards.head(10)

,playerID,awardID,yearID,lgID,tie,notes
2077,podrejo01,World Series MVP,1955,ML,NaN,NaN
2102,larsedo01,World Series MVP,1956,ML,NaN,NaN
2135,burdele01,World Series MVP,1957,ML,NaN,NaN
2177,turlebo01,World Series MVP,1958,ML,NaN,NaN
2219,sherrla01,World Series MVP,1959,ML,NaN,NaN
2263,richabo01,World Series MVP,1960,ML,NaN,NaN
2316,fordwh01,World Series MVP,1961,ML,NaN,NaN
2372,terryra01,World Series MVP,1962,ML,NaN,NaN
2427,koufasa01,World Series MVP,1963,ML,NaN,NaN
2481,gibsobo01,World Series MVP,1964,ML,NaN,NaN


So the 'World Series MVP' Awards has only existed since 1955.  Let's find out which teams won those years.

In [9]:
# Slim down to teams since 1955 and only those that won WS.
mvp_teams = teams[(teams['yearID'] > 1954) & (teams['WSWin'] == 1)]
mvp_teams.head(10)

,yearID,teamID,franchID,G,Ghome,W,L,LgWin,WSWin,R,...,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro,BA,OBP,SLG,OPS,WinP
1265,1955,BRO,LAD,154,77.0,98,55,1,1,857,...,104,101,BRO,BRO,BRO,0.270749,0.354525,0.448103,0.802629,0.640523
1290,1956,NYA,NYY,154,77.0,97,57,1,1,857,...,97,93,NYY,NYA,NYA,0.269767,0.345537,0.434111,0.779649,0.629870
1304,1957,ML1,ATL,155,78.0,95,59,1,1,772,...,93,90,MLN,MLN,MLN,0.269146,0.326069,0.441737,0.767805,0.616883
1321,1958,NYA,NYY,155,78.0,92,62,1,1,759,...,96,94,NYY,NYA,NYA,0.267850,0.335277,0.415565,0.750842,0.597403
1335,1959,LAN,LAD,156,78.0,88,68,1,1,705,...,107,107,LAD,LAN,LAN,0.257478,0.332198,0.396062,0.728260,0.564103
1355,1960,PIT,PIT,155,78.0,95,59,1,1,734,...,101,100,PIT,PIT,PIT,0.276175,0.335879,0.407140,0.743019,0.616883
1371,1961,NYA,NYY,163,81.0,109,53,1,1,827,...,95,93,NYY,NYA,NYA,0.262817,0.328417,0.441626,0.770043,0.672840
1390,1962,NYA,NYY,162,80.0,96,66,1,1,817,...,97,95,NYY,NYA,NYA,0.267364,0.336063,0.425939,0.762002,0.592593
1407,1963,LAN,LAD,163,81.0,99,63,1,1,640,...,93,91,LAD,LAN,LAN,0.250737,0.308451,0.356853,0.665304,0.611111
1435,1964,SLN,STL,162,81.0,93,69,1,1,715,...,109,108,STL,SLN,SLN,0.272178,0.323529,0.391822,0.715352,0.574074


In [10]:
# Let's Merge the two dataframes for simplicity.

mvp_join = mvp_awards.merge(mvp_teams, on='yearID').set_index('yearID')
mvp_join[['playerID', 'awardID', 'teamID', 'franchID', 'WinP']].head(10) # Slim to a few key columns for easy viewing.

,playerID,awardID,teamID,franchID,WinP
yearID,,,,,
1955,podrejo01,World Series MVP,BRO,LAD,0.640523
1956,larsedo01,World Series MVP,NYA,NYY,0.629870
1957,burdele01,World Series MVP,ML1,ATL,0.616883
1958,turlebo01,World Series MVP,NYA,NYY,0.597403
1959,sherrla01,World Series MVP,LAN,LAD,0.564103
1960,richabo01,World Series MVP,PIT,PIT,0.616883
1961,fordwh01,World Series MVP,NYA,NYY,0.672840
1962,terryra01,World Series MVP,NYA,NYY,0.592593
1963,koufasa01,World Series MVP,LAN,LAD,0.611111


Now we need to find out which teams these MVP players were on.

In [11]:
# Start with loading this csv of player Appearances
app = pd.read_csv('data/Appearances.csv') 

In [12]:
app.head(10)

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1871,TRO,NaN,abercda01,1,1.0,1,1.0,0,0,...,0,0,1,0,0,0,0,0.0,0.0,0.0
1,1871,RC1,NaN,addybo01,25,25.0,25,25.0,0,0,...,22,0,3,0,0,0,0,0.0,0.0,0.0
2,1871,CL1,NaN,allisar01,29,29.0,29,29.0,0,0,...,2,0,0,0,29,0,29,0.0,0.0,0.0
3,1871,WS3,NaN,allisdo01,27,27.0,27,27.0,0,27,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,1871,RC1,NaN,ansonca01,25,25.0,25,25.0,0,5,...,2,20,0,1,0,0,1,0.0,0.0,0.0
5,1871,FW1,NaN,armstbo01,12,12.0,12,12.0,0,0,...,0,0,0,0,11,1,12,0.0,0.0,0.0
6,1871,RC1,NaN,barkeal01,1,1.0,1,1.0,0,0,...,0,0,0,1,0,0,1,0.0,0.0,0.0
7,1871,BS1,NaN,barnero01,31,31.0,31,31.0,0,0,...,16,0,15,0,0,0,0,0.0,0.0,0.0
8,1871,FW1,NaN,barrebi01,1,1.0,1,1.0,0,1,...,0,1,0,0,0,0,0,0.0,0.0,0.0
9,1871,BS1,NaN,barrofr01,18,17.0,18,18.0,0,0,...,1,0,0,13,0,4,17,0.0,0.0,0.0


In [13]:
# Let's find where the 1955 MVP played.
app[(app['yearID'] == 1955) & (app['playerID'] == 'podrejo01')]

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
37875,1955,BRO,NL,podrejo01,32,24.0,32,27.0,27,0,...,0,0,0,0,0,0,0,0.0,1.0,4.0


In [14]:
# Who can forget Don Larsen throwing the only Perfect Game in WS History!
app[(app['yearID'] == 1956) & (app['playerID'] == 'larsedo01')]

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
38383,1956,NYA,AL,larsedo01,45,20.0,45,38.0,38,0,...,0,0,0,0,0,0,0,0.0,9.0,0.0


In [15]:
app[(app['yearID'] == 1957) & (app['playerID'] == 'burdele01')]

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
38763,1957,ML1,NL,burdele01,41,33.0,41,37.0,37,0,...,0,0,0,0,0,0,0,0.0,0.0,4.0


In [16]:
app[(app['yearID'] == 1958) & (app['playerID'] == 'turlebo01')]

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
39893,1958,NYA,AL,turlebo01,33,31.0,33,33.0,33,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [17]:
app[(app['yearID'] == 1959) & (app['playerID'] == 'sherrla01')]

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
40456,1959,LAN,NL,sherrla01,23,9.0,23,23.0,23,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [18]:
app[(app['yearID'] == 1960) & (app['playerID'] == 'richabo01')]

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
41063,1960,NYA,AL,richabo01,150,138.0,150,149.0,0,0,...,140,12,0,0,0,0,0,0.0,1.0,5.0


There!  So in 1960 the NY Yankees lost the World Series to Pittsburgh Pirates.  But the Series MVP was Bobby Richardson.